# Country-mean monthly temp and precip metrics
#### Christopher Callahan
#### Christopher.W.Callahan.GR@dartmouth.edu

#### Mechanics
Dependencies

In [1]:
import xarray as xr
import numpy as np
import sys
import os
import datetime
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, cm
from matplotlib import rcParams
import matplotlib.colors as colors
import matplotlib.gridspec as gridspec
import seaborn as sns
from rasterio import features
from affine import Affine
import geopandas as gp
import descartes
import cartopy.crs as ccrs
from cartopy.feature import ShapelyFeature
from scipy import stats
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

Data location

In [22]:
loc_shp = "../Data/ProcessedCountryShapefile/"
loc_pop = "../Data/GPW/"
loc_temp_best = "" #"/path/to/berkeley/earth/"
loc_precip = "" #"/path/to/gpcc/"
loc_out_temp = "../Data/CountryTemp/"
loc_out_precip = "../Data/CountryPrecip/"

Function for averaging

In [3]:
def transform_from_latlon(lat, lon):
    # Written by Alex Gottlieb
    lat = np.asarray(lat)
    lon = np.asarray(lon)
    trans = Affine.translation(lon[0], lat[0])
    scale = Affine.scale(lon[1] - lon[0], lat[1] - lat[0])
    return trans * scale

def rasterize_one(shape, latitude, longitude, fill=0, **kwargs):
    """Rasterize a shapefile geometry (polygon) onto the given
    xarray coordinates. This only works for 1d latitude and longitude
    arrays.
    Written by Alex Gottlieb and modified by Chris Callahan
    April 2020
    """
    transform = transform_from_latlon(latitude, longitude)
    out_shape = (len(latitude), len(longitude))
    raster = features.rasterize(shape, out_shape=out_shape,
                                fill=fill, transform=transform,
                                dtype=float, **kwargs)
    return xr.DataArray(raster, coords=[latitude,longitude], dims=["lat","lon"])

def rasterize(shapes, coords, fill=np.nan, **kwargs):
    """Rasterize a list of (geometry, fill_value) tuples onto the given
    xarray coordinates. This only works for 1d latitude and longitude
    arrays.
    """
    transform = transform_from_latlon(coords['lat'], coords['lon'])
    out_shape = (len(coords['lat']), len(coords['lon']))
    raster = features.rasterize(shapes, out_shape=out_shape,
                                fill=fill, transform=transform,
                                dtype=float, **kwargs)
    return xr.DataArray(raster, coords=coords, dims=('lat', 'lon'))

Functions for flipping longitude

In [4]:
def flip_lon_tll(da):
    # flip 360 to 180 lon
    # for time-lat-lon xarray dataarray

    # get coords
    lat_da = da.coords["lat"]
    lon_da = da.coords["lon"]
    time_da = da.coords["time"]

    # flip lon
    lon_180 = (lon_da.values + 180) % 360 - 180

    # new data array
    da_180 = xr.DataArray(da.values,
                          coords=[time_da,lat_da.values,lon_180],
                          dims=["time","lat","lon"])

    # flip dataarray so it goes from -180 to 180 
    # (instead of 0-180, -180-0)
    lon_min_neg = np.amin(lon_180[lon_180<0])
    lon_max_neg = np.amax(lon_180[lon_180<0])
    lon_min_pos = np.amin(lon_180[lon_180>=0])
    lon_max_pos = np.amax(lon_180[lon_180>=0])
    da_180_flip = xr.concat([da_180.loc[:,:,lon_min_neg:lon_max_neg],
                             da_180.loc[:,:,lon_min_pos:lon_max_pos]],
                            dim="lon")
    return(da_180_flip)

def flip_lon_ll(da):
    # flip 360 to 180 lon
    # for lat-lon xarray dataarray

    # get coords
    lat_da = da.coords["lat"]
    lon_da = da.coords["lon"]

    # flip lon
    lon_180 = (lon_da.values + 180) % 360 - 180

    # new data array
    da_180 = xr.DataArray(da.values,
                          coords=[lat_da,lon_180],
                          dims=["lat","lon"])

    # flip dataarray so it goes from -180 to 180 
    # (instead of 0-180, -180-0)
    lon_min_neg = np.amin(lon_180[lon_180<0])
    lon_max_neg = np.amax(lon_180[lon_180<0])
    lon_min_pos = np.amin(lon_180[lon_180>=0])
    lon_max_pos = np.amax(lon_180[lon_180>=0])
    da_180_flip = xr.concat([da_180.loc[:,lon_min_neg:lon_max_neg],
                             da_180.loc[:,lon_min_pos:lon_max_pos]],
                            dim="lon")
    return(da_180_flip)

#### Analysis

Population data -- may need multiple resolutions for different-resolution T and P

In [6]:
# half degree
pop_halfdegree = (xr.open_dataset(loc_pop+"30ArcMinute/gpw_v4_une_atotpopbt_cntm_30_min_lonflip.nc").data_vars["population"])[0,::-1,:]
pop_flip_halfdegree = flip_lon_ll(pop_halfdegree.rename({"latitude":"lat","longitude":"lon"}))
lat_min_halfdegree = np.amin(pop_flip_halfdegree.lat.values)
lat_max_halfdegree = np.amax(pop_flip_halfdegree.lat.values)

# one degree
pop_onedegree = (xr.open_dataset(loc_pop+"1Degree/gpw_v4_population_count_1degree_lonflip.nc").data_vars["population"])[0,::-1,:]
pop_flip_onedegree = flip_lon_ll(pop_onedegree.rename({"latitude":"lat","longitude":"lon"}))
lat_min_onedegree = np.amin(pop_flip_onedegree.lat.values)
lat_max_onedegree = np.amax(pop_flip_onedegree.lat.values)

Shapefile

In [7]:
shp = gp.read_file(loc_shp)
iso_shp = shp.ISO3.values
isonums = {i: k for i, k in enumerate(shp.ISO3)}
isonums_rev = {k: i for i, k in enumerate(shp.ISO3)}
shapes = [(shape, n) for n, shape in enumerate(shp.geometry)]

Read in temperature

In [8]:
y1_temp = 1900
y2_temp = 2019

BEST

In [9]:
y1_best = 1850
y2_best = 2022
tmean_best = xr.open_dataset(loc_temp_best+"BerkeleyEarth_Land_and_Ocean_LatLong1.nc").rename({"latitude":"lat","longitude":"lon"})
tmean_anom = tmean_best.temperature
tmean_clm = tmean_best.climatology
lsm1 = tmean_best.land_mask
lsm = lsm1.where(lsm1==1,np.nan)

In [10]:
#tmean_anom = xr.open_dataset(loc_temp_best+"be_avg_temp_land_ocean.nc").temperature.rename({"latitude":"lat","longitude":"lon"})

In [11]:
tmean_time_new = pd.date_range(start=str(y1_temp)+"-01-01",end=str(y2_temp)+"-12-31",freq="MS")
tmean_time_round = np.floor(tmean_anom.time.values)
tmean_anom_yrs = tmean_anom[(tmean_time_round>=y1_temp)&(tmean_time_round<(y2_temp+1))]
tmean_anom_yrs.coords["time"] = tmean_time_new

tmean_be = xr.DataArray(np.full(tmean_anom_yrs.shape,np.nan),
                     coords=[tmean_time_new,tmean_anom_yrs.coords["lat"],tmean_anom_yrs.coords["lon"]],
                     dims=["time","lat","lon"])
for mm in np.arange(0,12,1):
    print(mm)
    indices = tmean_time_new.month==(mm+1)
    tmean_be[indices,:,:] = tmean_anom_yrs[indices,:,:] + tmean_clm[mm,:,:]
    
tmean_best_foravg = tmean_be.loc[:,lat_min_onedegree:lat_max_onedegree,:] #*lsm

0
1
2
3
4
5
6
7
8
9
10
11


In [12]:
tmean = tmean_best_foravg*1.0

Read in GPCP precip

In [13]:
precip = xr.open_dataset(loc_precip+"precip.mon.total.v2020.nc").precip

Set years

In [14]:
#y1_temp = 1979
#y2_temp = 2018
y1_precip = 1900
y2_precip = 2019

In [16]:
precip_yrs = precip[:,::-1,:].loc[str(y1_precip)+"-01-01":str(y2_precip)+"-12-31",lat_min_halfdegree:lat_max_halfdegree,:]
precip_flip = flip_lon_tll(precip_yrs)

In [17]:
def xr_country_average(data,y1,y2,freq,shapes,iso_list,isonums_dict,weight=False,weightdata=None):
    time = pd.date_range(start=str(y1)+"-01-01",end=str(y2)+"-12-31",freq=freq)
    country_array = xr.DataArray(np.full((len(iso_list),len(time)),np.nan),
                                 coords=[iso_list,time],
                                 dims=["iso","time"])
    
    isoraster = rasterize(shapes,data.drop("time").coords)
    data.coords["country"] = isoraster
    if weight:
        weightdata.coords["country"] = isoraster
        countrymean = ((data * weightdata).groupby("country").sum())/(weightdata.groupby("country").sum())
    else:
        countrymean = data.groupby("country").mean()
    
    isocoord = np.array([isonums_dict[n] for n in countrymean.coords["country"].values])
    country_array.loc[isocoord,:] = countrymean.transpose("country","time").values
    return(country_array)

In [18]:
tmean_country = xr_country_average(tmean,y1_temp,y2_temp,"MS",
                                  shapes,iso_shp,isonums,True,pop_flip_onedegree)
#tmean_country = tmean_country - 273.15

precip_country = xr_country_average(precip_flip,y1_precip,y2_precip,"MS",
                                  shapes,iso_shp,isonums,True,pop_flip_halfdegree) 

In [19]:
tmean_country_anom = (tmean_country.groupby("time.month") - (tmean_country.groupby("time.month").mean(dim="time")))
tmean_country_std = tmean_country_anom.groupby("time.month")/tmean_country.groupby("time.month").std(dim="time")

precip_country_anom = (precip_country.groupby("time.month") - (precip_country.groupby("time.month").mean(dim="time")))
precip_country_std = precip_country_anom.groupby("time.month")/precip_country.groupby("time.month").std(dim="time")

/dartfs-hpc/rc/home/y/f003k8y/.conda/envs/ccallahan/lib/python3.7/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/dartfs-hpc/rc/home/y/f003k8y/.conda/envs/ccallahan/lib/python3.7/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [20]:
tmean_country.name = "temp"
tmean_country.attrs["creation_date"] = str(datetime.datetime.now())
tmean_country.attrs["created_by"] = "Christopher Callahan, Christopher.W.Callahan.GR@dartmouth.edu"
tmean_country.attrs["variable_description"] = "Monthly average temperature at the country level, BEST/20CR/UDel"
tmean_country.attrs["created_from"] = os.getcwd()+"/Process_Country_TempPrecip.ipynb"
tmean_country.attrs["coords"] = "iso x time"
tmean_country.attrs["units"] = "Degrees C"

fname_out = loc_out_temp+"BerkeleyEarth_country_temp_monthly_"+str(y1_temp)+"-"+str(y2_temp)+".nc"
tmean_country.to_netcdf(fname_out,mode="w")
print(fname_out)

tmean_country_std.name = "temp"
tmean_country_std.attrs["creation_date"] = str(datetime.datetime.now())
tmean_country_std.attrs["created_by"] = "Christopher Callahan, Christopher.W.Callahan.GR@dartmouth.edu"
tmean_country_std.attrs["variable_description"] = "Standardized monthly average temperature at the country level, BEST/20CR/UDe;"
tmean_country_std.attrs["created_from"] = os.getcwd()+"/Process_Country_TempPrecip.ipynb"
tmean_country_std.attrs["coords"] = "iso x time"
tmean_country_std.attrs["units"] = "standard deviations"

fname_out = loc_out_temp+"BerkeleyEarth_country_temp_monthly_std_"+str(y1_temp)+"-"+str(y2_temp)+".nc"
tmean_country_std.to_netcdf(fname_out,mode="w")
print(fname_out)

../Data/CountryTemp/BerkeleyEarth_country_temp_monthly_1900-2019.nc
../Data/CountryTemp/BerkeleyEarth_country_temp_monthly_std_1900-2019.nc


In [21]:
precip_country.name = "precip"
precip_country.attrs["creation_date"] = str(datetime.datetime.now())
precip_country.attrs["created_by"] = "Christopher Callahan, Christopher.W.Callahan.GR@dartmouth.edu"
precip_country.attrs["variable_description"] = "Monthly total precip, averaged over country"
precip_country.attrs["created_from"] = os.getcwd()+"/Process_Country_TempPrecip.ipynb"
precip_country.attrs["coords"] = "iso x time"
precip_country.attrs["units"] = "millimeters"

fname_out = loc_out_precip+"GPCC_country_precip_monthly_"+str(y1_precip)+"-"+str(y2_precip)+".nc"
precip_country.to_netcdf(fname_out,mode="w")
print(fname_out)

precip_country_std.name = "precip"
precip_country_std.attrs["creation_date"] = str(datetime.datetime.now())
precip_country_std.attrs["created_by"] = "Christopher Callahan, Christopher.W.Callahan.GR@dartmouth.edu"
precip_country_std.attrs["variable_description"] = "Standardized monthly total precip, averaged over country"
precip_country_std.attrs["created_from"] = os.getcwd()+"/Process_Country_TempPrecip.ipynb"
precip_country_std.attrs["coords"] = "iso x time"
precip_country_std.attrs["units"] = "standard deviations"

fname_out = loc_out_precip+"GPCC_country_precip_monthly_std_"+str(y1_precip)+"-"+str(y2_precip)+".nc"
precip_country_std.to_netcdf(fname_out,mode="w")
print(fname_out)

../Data/CountryPrecip/GPCC_country_precip_monthly_1900-2019.nc
../Data/CountryPrecip/GPCC_country_precip_monthly_std_1900-2019.nc
